
# CISC 610-51-A
# Data Structures & Algorithms
## Final Project
## Matthew Sabo
***

### Introduction:
With this project, I aim to create a program that will be able to manage various tasks and assignments using a Priority Queue. The program should be able to add an assignment and place it in the right place in the queue, remove the current assignment from the queue, list the current assignment that the user should be working on, and remind the user of when a task or assignment is due.


### Data Structures and Algorithms Used:
##### Linked List:
The reason why a linked list was used in this project is because of the ease of implementation and the time efficiency compared to other options like an array which is less time efficient in removal and peek functions being O(n) compared to a linked list which is O(1). A Binary heap would be more time efficient than a linked list for this implementation, but it is more difficult to implement than using a linked list. So the ease of implementation and the time efficiency compared to other data structures is why I decided to use a linked list.
##### Priority Queue:     
The reason why I decided to use a Priority Queue compared to other data structures is because it allows for the priority of the assignment to be a factor in where a new node would be placed in the queue rather than a regular queue which would place the new assignment/task on the bottom of the queue regardless of the importance or due date of the assignment/task.
***

This first block of code defines the main data structure that I will use for this assignment, being a Priority Queue. The PriorityQueue class has 5 functions: isEmpty which checks if the linked list is empty, push adds a node to the linked list in the correct position based on the priority of the new node, pop removes the head node from the list and makes the next node the new head node, traverse traverses the linked list and prints out the assignment name and due date, and peek returns the head node. This block also includes a Node class which stores the information of the node and the priority that will be used in calculating where it is placed inside of the linked list

In [5]:
from datetime import datetime, timedelta

class Node:
    def __init__(self, data, priority):
        self.data = data
        self.priority = priority
        self.next = None

class PriorityQueue:
    def __init__(self):
        self.head = None
        
    def isEmpty(self):
        if self.head == None:
            return True
        else:
            return False
    
    def push(self, value, priority):
        if self.isEmpty() == True:
            self.head = Node(value, priority)
            return
        else:
            # Compares the priority of the new node compared to the head to determine
            # if it should be the new head node
            if self.head.priority < priority:
                # The new node becomes the head node
                newNode = Node(value, priority)
                newNode.next = self.head
                self.head = newNode
            else:
                # The new nodes priority is less than the head node and it traverses the 
                # linked list until it finds its spot
                tempNode = self.head
                while tempNode.next:
                    # Breaks the loop if it finds its correct spot
                    if priority >= tempNode.next.priority:
                        break
                    tempNode = tempNode.next
                newNode = Node(value, priority)
                newNode.next = tempNode.next
                tempNode.next = newNode
    
    def pop(self):
        if self.isEmpty() is True:
            return
        else:
            oldHead = self.head 
            self.head = self.head.next
            return oldHead
    
    def peek(self):
        return self.head
    
    def traverse(self):
        temp = []
        if self.isEmpty() is True:
            print("No Assignments in Queue\n")
            return
        else:
            currNode = self.head
            while currNode:
                # Prints out the name and when the assignments are due
                print(currNode.data.name,"due at", datetime.strftime(currNode.data.deadline, "%m-%d-%Y %I:%M %p"), end="\n")
                currNode = currNode.next

The Assignment class is used to store the name and the deadline of the assignment. Its different than the node class because the node class only needs the priority and the data. This class is used to hold all of the information of the assignment, like the due date and the name of the assignment, and calculate the priority of the assignment since it takes the priority that is inputted by the user and compares that to the amount of time that the user has until the assignment is due to create the priority used when pushing a new node on top of the stack.


In [6]:
class Assignment:
    def __init__(self, name, deadline, importance):
        self.name = name
        self.deadline = deadline
        self.importance = importance
        
    def priority(self):
        time_to_deadline = (self.deadline - datetime.now()).total_seconds()
        # Max of 24 hours divided by the time_to_deadline
        urgency = max(1, 86400 / time_to_deadline)
        return urgency * self.importance


The Study Scheduler class is where most of the logic happens. It starts out by initializing a priority queue and then it contains functions to add an assignment to the queue, complete an assignment and remove it from the queue, show what assignments are in the queue, remind the user what the current assignment they should be working on, and shows the user how much time they have left to work on an assignment.  

In [7]:
class StudyScheduler:
    def __init__(self):
        self.assignments = PriorityQueue()
    
    def add_assignment(self, name, deadline, importance):
        # Takes the due date of the assignment and puts it in the proper format
        deadlineDateTime = datetime.strptime(deadline, "%m-%d-%Y %I:%M %p")
        # Creates the assignment and then pushes it onto the queue
        assignment = Assignment(name, deadlineDateTime, importance)
        self.assignments.push(assignment, assignment.priority())
        
    def complete_assignment(self):
        if not self.assignments.isEmpty():
            assignment = self.assignments.pop()
            print("\nAssignment", assignment.data.name, "marked as complete")
        else:
            print('No tasks available to complete')
            
    def show_assignments(self):
        print("\nUpcoming Assignments:")
        self.assignments.traverse()
        
    def current_assignment(self):
        currAssignment = self.assignments.peek()
        print("\nCurrent Assignment: ")
        print(currAssignment.data.name, "due at", datetime.strftime(currAssignment.data.deadline, "%m-%d-%Y %I:%M %p"))
        
    def reminder(self):
        currNode = self.assignments.head
        while currNode:
            # Calculates the total time left in seconds and then warns the user when an assignment is due as long as it 
            # is due in less than 12 hours
            time_left = (currNode.data.deadline - datetime.now()).total_seconds()
            timeLeftDelta = str(timedelta(seconds=time_left))
            hours_mins = timeLeftDelta.split(':')
            match time_left:
                case time_left if time_left < 43200:
                    # Less than 12 hours remaining
                    print(f"\nReminder: Assignment", currNode.data.name, 'is due in', hours_mins[0], 'hours and', hours_mins[1], 'minutes!')
            currNode = currNode.next


Initializes a StudyScheduler and ensures all of the functions in the class works. The while True loop is designed to keep the program running until the user is finished using it. It allows the user to add an assignment, show all assignments in the queue, complete an assignment, remind the user about when an assignment is due, and also exit the program. Underneath the while True loop, I have an example of how multiple assignments work in the program, and to show that the Priority Queue works correctly in regards to sorting the queue based on priority.

In [8]:
study_scheduler = StudyScheduler()

while True:
    task = input("What would you like to do? (Add Assignments, Show Queue, Complete Assignment, Current Assignment, Reminder, Exit) : ")
    
    match task:
        case "Add Assignments":
            assignment_name = input("Please input assignment name: ")
            assignment_date = input("Please input assignment date (format:M-D-Y H:M AM/PM): ")
            assignment_priority = input("Please input assignment priority from a scale of 1-10: ")
            study_scheduler.add_assignment(assignment_name, assignment_date, int(assignment_priority))
        case "Show Queue":
            study_scheduler.show_assignments()
        case "Complete Assignment":
            study_scheduler.complete_assignment()
        case "Current Assignment":
            study_scheduler.current_assignment()
        case "Reminder":
            study_scheduler.reminder()
        case "Exit":
            break

# Simulation of someone using the program. Change simuTest to false to run the program as intended
simuTest = True

if simuTest is True:
    study_scheduler.add_assignment("Final Exam", "11-29-2024 11:59 PM", 5)
    study_scheduler.add_assignment("Math Homework", "11-27-2024 10:59 PM", 6)
    study_scheduler.add_assignment("Final Presentation", "11-27-2024 10:59 PM", 8)
    study_scheduler.add_assignment("Study for final exam", "11-28-2024 12:49 PM", 5)
    study_scheduler.add_assignment("Formal Automata Theory Lab", "11-27-2024 10:59 PM", 10)

    study_scheduler.show_assignments()

    study_scheduler.complete_assignment()
    study_scheduler.current_assignment()
    study_scheduler.complete_assignment()

    study_scheduler.show_assignments()

    study_scheduler.reminder()


Upcoming Assignments:
Test due at 11-27-2024 10:00 PM

Current Assignment: 
Test due at 11-27-2024 10:00 PM

Assignment Test marked as complete

Upcoming Assignments:
Formal Automata Theory Lab due at 11-27-2024 10:59 PM
Final Presentation due at 11-27-2024 10:59 PM
Math Homework due at 11-27-2024 10:59 PM
Study for final exam due at 11-28-2024 12:49 PM
Final Exam due at 11-29-2024 11:59 PM

Assignment Formal Automata Theory Lab marked as complete

Current Assignment: 
Final Presentation due at 11-27-2024 10:59 PM

Assignment Final Presentation marked as complete

Upcoming Assignments:
Math Homework due at 11-27-2024 10:59 PM
Study for final exam due at 11-28-2024 12:49 PM
Final Exam due at 11-29-2024 11:59 PM

Reminder: Assignment Math Homework is due in 1 hours and 55 minutes!
